In [2]:
# imports
import logging
import mdtraj as md
import numpy as np
import os
import shutil
import pickle
import pandas as pd

from openeye import oechem
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from simtk import unit

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:rdkit:Enabling RDKit 2020.03.4 jupyter extensions


In [3]:
def render_protein_residue_atom_mapping(topology_proposal, filename):
    """
    wrap the `render_atom_mapping` method around protein point mutation topologies.
    TODO : make modification to `render_atom_mapping` so that the backbone atoms are not written in the output.
    
    arguments
        topology_proposal : perses.rjmc.topology_proposal.TopologyProposal object
            topology proposal of protein mutation
        filename : str
            filename to write the map
    """
    from perses.utils.smallmolecules import render_atom_mapping
    oe_res_maps = {}
    for omm_new_idx, omm_old_idx in topology_proposal._new_to_old_atom_map.items():
        if omm_new_idx in topology_proposal._new_topology.residue_to_oemol_map.keys():
            try:
                oe_res_maps[topology_proposal._new_topology.residue_to_oemol_map[omm_new_idx]] = topology_proposal._old_topology.residue_to_oemol_map[omm_old_idx]
            except:
                pass
            
    render_atom_mapping(filename, topology_proposal._old_topology.residue_oemol, topology_proposal._new_topology.residue_oemol, oe_res_maps)

In [4]:
# Create mutation output directory
output_prefix = "./mutations_output/"
os.makedirs(output_prefix, exist_ok=True)
print("--> Directory ", output_prefix, " created ")

--> Directory  ./mutations_output/  created 


In [5]:
df = pd.read_csv('../data/hauser_mutations.csv', delimiter=',', decimal=",")

In [6]:
df

,tki,mutation,resid,from,to,IC50,ddG_exp
0,axitinib,M244V,244,MET,VAL,690,-0.11
1,axitinib,L248R,248,LEU,ARG,1393,0.31
2,axitinib,L248V,248,LEU,VAL,1399,0.32
3,axitinib,G250E,250,GLY,GLU,1295,0.27
4,axitinib,Q252H,252,GLN,HIS,1155,0.2
...,...,...,...,...,...,...,...
139,ponatinib,F359C,359,PHE,CYS,6,0.41
140,ponatinib,F359I,359,PHE,ILE,11,0.77
141,ponatinib,F359V,359,PHE,VAL,4,0.17
142,ponatinib,H396R,396,HIS,ARG,4,0.17


In [7]:
ls ../data/abl_complexes/

2f4j_gefitinib.sdf  2hiw.pdb            3cs9_nilotinib.sdf  4twp_axitinib.sdf
2f4j.pdb            2hiw_ponatinib.sdf  3cs9.pdb            4twp.pdb
2g1t_erlotinib.sdf  2hyy_imatinib.sdf   3ue4_bosutinib.sdf  4xey_dasatinib.sdf
2g1t.pdb            2hyy.pdb            3ue4.pdb            4xey.pdb


In [20]:
index_dict = {0: ['gefitinib', '2f4j'],
              1: ['erlotinib', '2g1t'],
              2: ['ponatinib', '2hiw'],
              3: ['imatinib', '2hyy'],
              4: ['nilotinib', '3cs9'],
              5: ['bosutinib', '3ue4'],
              6: ['axitinib', '4twp'],
              7: ['dasatinib', '4xey']            
           }

In [21]:
i = 0

In [22]:
index_dict[i]

['gefitinib', '2f4j']

In [24]:
df_i = df[df['tki'] == index_dict[i][0]]

In [25]:
df_i

,tki,mutation,resid,from,to,IC50,ddG_exp
75,gefitinib,Q252H,252,GLN,HIS,230,-0.44
76,gefitinib,Y253F,253,TYR,PHE,360,-0.17
77,gefitinib,E255K,255,GLU,LYS,400,-0.11
78,gefitinib,F317I,317,PHE,ILE,4700,1.35
79,gefitinib,F317L,317,PHE,LEU,780,0.29
80,gefitinib,M351T,351,MET,THR,520,0.05


In [26]:
data_path = "../data/abl_complexes/"
out_dir = './mutations_output/'

aa = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}



for index, row in df_i.iterrows():
    
    print(row['tki'])

    ligand_name = row['tki']
    protein_name = lig_prot[ligand_name]
    resid = str(row['resid'])  # The resid has to be a string for PointMutationExecutor input
    mutate_from = row['from']
    mutate_to = row['to']
    
    # Create protein directory
    output_prefix_protein = f"./mutations_output/{protein_name}"
    if not os.path.exists(output_prefix_protein):
        os.makedirs(output_prefix_protein)
        print(f"--> Directory {output_prefix_protein} created")
    
    # Create protein mutant directory
    output_prefix_mutant = f"./mutations_output/{protein_name}/{aa[mutate_from]}{resid}{aa[mutate_to]}"
    if not os.path.exists(output_prefix_mutant):
        os.makedirs(output_prefix_mutant)
        print(f"--> Directory {output_prefix_mutant} created")

    # make the system
    solvent_delivery = PointMutationExecutor(f"{data_path}{protein_name}.pdb", 
                        '1', # First and only protein chain 
                        resid, 
                        mutate_to,
                        ligand_file=f"{data_path}{protein_name}_{ligand_name}.sdf",
                        ionic_strength=0.15*unit.molar
                       )
    
    # make image map of the transformation
    render_protein_residue_atom_mapping(solvent_delivery.get_apo_htf()._topology_proposal,
                                        f"{output_prefix_mutant}/{protein_name}_{ligand_name}_{aa[mutate_from]}{resid}{aa[mutate_to]}_apo_map.png")

    # pickle the output and save
    pickle.dump(solvent_delivery.get_apo_htf(),
                open(os.path.join(output_prefix_mutant,
                                  f"{protein_name}_{ligand_name}_{aa[mutate_from]}{resid}{aa[mutate_to]}_apo.pickle"), "wb" ))

    pickle.dump(solvent_delivery.get_complex_htf(),
                open(os.path.join(output_prefix_mutant,
                                  f"{protein_name}_{ligand_name}_{aa[mutate_from]}{resid}{aa[mutate_to]}_complex.pickle"), "wb" ))


    # save the coordinates to of apo and complex to check the geometry of the transformation
    htfs_t = [solvent_delivery.get_apo_htf(), solvent_delivery.get_complex_htf()]

    top_old = md.Topology.from_openmm(htfs_t[0]._topology_proposal.old_topology)
    top_new = md.Topology.from_openmm(htfs_t[0]._topology_proposal.new_topology)
    traj = md.Trajectory(np.array(htfs_t[0].old_positions(htfs_t[0].hybrid_positions)), top_old)
    traj.save(f"{output_prefix_mutant}/{protein_name}_{ligand_name}_{aa[mutate_from]}{resid}{aa[mutate_to]}_apo_old.pdb")
    traj = md.Trajectory(np.array(htfs_t[0].new_positions(htfs_t[0].hybrid_positions)), top_new)
    traj.save(f"{output_prefix_mutant}/{protein_name}_{ligand_name}_{aa[mutate_from]}{resid}{aa[mutate_to]}_apo_new.pdb")

    top_old = md.Topology.from_openmm(htfs_t[1]._topology_proposal.old_topology)
    top_new = md.Topology.from_openmm(htfs_t[1]._topology_proposal.new_topology)
    traj = md.Trajectory(np.array(htfs_t[1].old_positions(htfs_t[1].hybrid_positions)), top_old)
    traj.save(f"{output_prefix_mutant}/{protein_name}_{ligand_name}_{aa[mutate_from]}{resid}{aa[mutate_to]}_complex_old.pdb")
    traj = md.Trajectory(np.array(htfs_t[1].new_positions(htfs_t[1].hybrid_positions)), top_new)
    traj.save(f"{output_prefix_mutant}/{protein_name}_{ligand_name}_{aa[mutate_from]}{resid}{aa[mutate_to]}_complex_new.pdb")

gefitinib
--> Directory ./mutations_output/2f4j created
--> Directory ./mutations_output/2f4j/Q252H created


INFO:utils.openeye:molecule _1                      does not have unique atom names. Generating now...
INFO:utils.openeye:molecule _1                      has unique atom names already
INFO:root:solvating at 0.15 M using tip3p


KeyboardInterrupt: 